In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn.utils.prune as prune
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import os

# Define dataset transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Download and Load TensorFlow Flowers Dataset
data_dir = "./data/tf_flowers"

if not os.path.exists(data_dir):
    os.system(f"wget http://download.tensorflow.org/example_images/flower_photos.tgz -O flowers.tgz")
    os.system(f"mkdir -p {data_dir} && tar -xvf flowers.tgz -C {data_dir} --strip-components=1")

# Load dataset using ImageFolder
dataset = ImageFolder(root=data_dir, transform=transform)

# Split into train and validation sets (80%-20%)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Load Pretrained ResNet-50 Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=True)

# Modify the last FC layer for 5 flower classes
num_classes = len(dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# Train function
def train_model(model, dataloader, optimizer, criterion, epochs=5):
    model.train()
    for epoch in range(epochs):
        total_loss, correct, total = 0, 0, 0
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader):.4f}, Accuracy: {correct/total:.4f}")

# Train before pruning
print("Training Base Model...")
train_model(model, train_loader, optimizer, criterion, epochs=5)

# ----------------- STRUCTURED PRUNING -----------------

def prune_model(model, amount=0.3):
    """
    Apply structured pruning (L1 Norm based) on convolutional layers.
    Removes `amount` of least important filters per layer.
    """
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d):
            prune.ln_structured(module, name='weight', amount=amount, n=1, dim=0)  # L1 Norm Pruning
            prune.remove(module, 'weight')  # Make pruning permanent
    return model

# Apply structured pruning (30% of filters)
print("Applying Structured Pruning...")
pruned_model = prune_model(model, amount=0.3)

# Fine-tune after pruning
print("Fine-tuning Pruned Model...")
train_model(pruned_model, train_loader, optimizer, criterion, epochs=5)

# Evaluate the model
def evaluate_model(model, dataloader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    return correct / total

# Evaluate on validation set
accuracy = evaluate_model(pruned_model, val_loader)
print(f"Final Pruned Model Accuracy: {accuracy:.4f}")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 54.2MB/s]


Training Base Model...
Epoch 1, Loss: 0.3956, Accuracy: 0.8685
Epoch 2, Loss: 0.0991, Accuracy: 0.9680
Epoch 3, Loss: 0.0605, Accuracy: 0.9823
Epoch 4, Loss: 0.0457, Accuracy: 0.9867
Epoch 5, Loss: 0.0314, Accuracy: 0.9918
Applying Structured Pruning...
Fine-tuning Pruned Model...
Epoch 1, Loss: 0.4585, Accuracy: 0.8471
Epoch 2, Loss: 0.1566, Accuracy: 0.9489
Epoch 3, Loss: 0.0947, Accuracy: 0.9714
Epoch 4, Loss: 0.0746, Accuracy: 0.9782
Epoch 5, Loss: 0.0458, Accuracy: 0.9837
Final Pruned Model Accuracy: 0.9237
